## Import Libraries

In [17]:
import pandas as pd
import numpy as np

import math

import os
import sys

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

## Data

In [267]:
# Paths for data.
Ravdess = "input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"
Crema = "input/cremad/AudioWAV/"
Tess = "input/Tess/Tess/"
Savee = "input/surrey-audiovisual-expressed-emotion-savee/ALL/"

In [268]:
ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(Ravdess + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

,Emotions,Path
0,angry,input/ravdess-emotional-speech-audio/audio_spe...
1,fear,input/ravdess-emotional-speech-audio/audio_spe...
2,fear,input/ravdess-emotional-speech-audio/audio_spe...
3,angry,input/ravdess-emotional-speech-audio/audio_spe...
4,disgust,input/ravdess-emotional-speech-audio/audio_spe...


In [269]:
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []

for file in crema_directory_list:
    # storing file paths
    file_path.append(Crema + file)
    # storing file emotions
    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df.head()

,Emotions,Path
0,angry,input/cremad/AudioWAV/1022_ITS_ANG_XX.wav
1,angry,input/cremad/AudioWAV/1037_ITS_ANG_XX.wav
2,neutral,input/cremad/AudioWAV/1060_ITS_NEU_XX.wav
3,neutral,input/cremad/AudioWAV/1075_ITS_NEU_XX.wav
4,disgust,input/cremad/AudioWAV/1073_IOM_DIS_XX.wav


In [270]:
tess_directory_list = os.listdir(Tess)

file_emotion = []
file_path = []

for dir in tess_directory_list:
    directories = os.listdir(Tess + dir)
    for file in directories:
        part = file.split('.')[0]
        part = part.split('_')[2]
        if part=='ps':
            file_emotion.append('surprise')
        else:
            file_emotion.append(part)
        file_path.append(Tess + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Tess_df = pd.concat([emotion_df, path_df], axis=1)
Tess_df.head()

,Emotions,Path
0,disgust,input/Tess/Tess/YAF_disgust/YAF_date_disgust.wav
1,disgust,input/Tess/Tess/YAF_disgust/YAF_rag_disgust.wav
2,disgust,input/Tess/Tess/YAF_disgust/YAF_raise_disgust.wav
3,disgust,input/Tess/Tess/YAF_disgust/YAF_ditch_disgust.wav
4,disgust,input/Tess/Tess/YAF_disgust/YAF_door_disgust.wav


In [271]:
savee_directory_list = os.listdir(Savee)

file_emotion = []
file_path = []

for file in savee_directory_list:
    file_path.append(Savee + file)
    part = file.split('_')[1]
    ele = part[:-6]
    if ele=='a':
        file_emotion.append('angry')
    elif ele=='d':
        file_emotion.append('disgust')
    elif ele=='f':
        file_emotion.append('fear')
    elif ele=='h':
        file_emotion.append('happy')
    elif ele=='n':
        file_emotion.append('neutral')
    elif ele=='sa':
        file_emotion.append('sad')
    else:
        file_emotion.append('surprise')
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Savee_df = pd.concat([emotion_df, path_df], axis=1)
Savee_df.head()

,Emotions,Path
0,sad,input/surrey-audiovisual-expressed-emotion-sav...
1,sad,input/surrey-audiovisual-expressed-emotion-sav...
2,neutral,input/surrey-audiovisual-expressed-emotion-sav...
3,surprise,input/surrey-audiovisual-expressed-emotion-sav...
4,neutral,input/surrey-audiovisual-expressed-emotion-sav...


In [272]:
data_path = pd.concat([Ravdess_df, Crema_df, Tess_df, Savee_df], axis = 0)
data_path.to_csv("data_path.csv",index=False)
data_path.head()

,Emotions,Path
0,angry,input/ravdess-emotional-speech-audio/audio_spe...
1,fear,input/ravdess-emotional-speech-audio/audio_spe...
2,fear,input/ravdess-emotional-speech-audio/audio_spe...
3,angry,input/ravdess-emotional-speech-audio/audio_spe...
4,disgust,input/ravdess-emotional-speech-audio/audio_spe...


In [273]:
df_list_by_emotions = []
emotion_list = ["angry","fear","disgust","sad","surprise","happy","calm","neutral"]
for emotion in emotion_list:
    df_list_by_emotions.append(data_path.loc[data_path["Emotions"] == emotion])

In [54]:
#12162 datapoints
sample_rate = 0.3
sample_path_list = []
for df in df_list_by_emotions:
    sample_path_list.append(df.sample(frac = sample_rate))

In [133]:
sample_path_and_label = pd.concat(sample_path_list, axis=0)
sample_path_and_label.reset_index(inplace=True)
sample_path_and_label.drop("index",axis=1, inplace=True)
sample_path_and_label

,Emotions,Path
0,angry,input/cremad/AudioWAV/1027_IOM_ANG_XX.wav
1,angry,input/cremad/AudioWAV/1038_IOM_ANG_XX.wav
2,angry,input/Tess/Tess/YAF_angry/YAF_chief_angry.wav
3,angry,input/Tess/Tess/YAF_angry/YAF_time_angry.wav
4,angry,input/cremad/AudioWAV/1089_TIE_ANG_XX.wav
...,...,...
3645,neutral,input/Tess/Tess/OAF_neutral/OAF_life_neutral.wav
3646,neutral,input/cremad/AudioWAV/1012_MTI_NEU_XX.wav
3647,neutral,input/cremad/AudioWAV/1002_ITH_NEU_XX.wav
3648,neutral,input/cremad/AudioWAV/1009_IOM_NEU_XX.wav


## Label Data

In [4]:
sample_data = pd.read_csv("sample_audio_data.csv")
sample_data

,0,1,2,3,4,5,6,7,8,9,...,28394,28395,28396,28397,28398,28399,28400,28401,Path,zcr
0,-0.019919,-0.018074,-0.016522,-0.016333,-0.016760,-0.015943,-0.013925,-0.010346,-0.004805,-0.000656,...,0.058045,0.045831,0.025694,0.005988,-0.012835,-0.032857,-0.046770,-0.056439,input/cremad/AudioWAV/1027_IOM_ANG_XX.wav,0.094343
1,-0.066534,-0.063018,-0.065487,-0.065889,-0.058385,-0.046553,-0.033967,-0.021541,-0.013983,-0.011537,...,-0.080126,-0.075396,-0.081968,-0.101987,-0.136541,-0.191647,-0.239525,-0.256562,input/cremad/AudioWAV/1038_IOM_ANG_XX.wav,0.071437
2,0.018359,0.039753,0.063115,0.051443,0.004972,-0.056535,-0.070990,-0.085613,-0.054105,0.031816,...,-0.156671,-0.012520,0.132811,0.105119,-0.036655,-0.076660,0.035627,0.137454,input/Tess/Tess/YAF_angry/YAF_chief_angry.wav,0.157532
3,0.073088,0.068950,0.092506,0.107812,0.100205,0.068652,0.055896,0.054489,0.035362,-0.000391,...,-0.041145,-0.036965,-0.037242,-0.043026,-0.055801,-0.069270,-0.086586,-0.100614,input/Tess/Tess/YAF_angry/YAF_time_angry.wav,0.127040
4,-0.035151,0.025190,0.062018,0.056600,0.024191,-0.002286,-0.005931,-0.006781,-0.022146,-0.026168,...,-0.208820,-0.153628,-0.061097,-0.082004,-0.210577,-0.213449,-0.042307,0.175471,input/cremad/AudioWAV/1089_TIE_ANG_XX.wav,0.176331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3645,-0.016913,-0.021947,-0.026967,-0.030889,-0.031055,-0.030822,-0.030577,-0.029294,-0.030711,-0.032197,...,-0.018098,-0.017201,-0.017159,-0.014812,-0.010597,-0.007876,-0.001778,0.002326,input/Tess/Tess/OAF_neutral/OAF_life_neutral.wav,0.033892
3646,0.003167,0.000837,0.000244,-0.003053,-0.006417,-0.006227,-0.005134,-0.004948,-0.004400,-0.004328,...,0.028449,0.028072,0.027727,0.026263,0.024564,0.023976,0.023458,0.022242,input/cremad/AudioWAV/1012_MTI_NEU_XX.wav,0.057251
3647,0.009664,-0.006643,-0.014054,-0.016295,-0.021871,-0.030279,-0.041057,-0.048791,-0.050589,-0.055038,...,-0.001477,-0.001414,-0.000838,-0.000476,-0.000737,-0.000814,-0.000886,-0.000922,input/cremad/AudioWAV/1002_ITH_NEU_XX.wav,0.064654
3648,0.001670,0.001173,0.000745,0.000433,-0.000045,-0.000716,-0.000934,-0.001110,-0.002230,-0.003157,...,0.000721,-0.002279,-0.005768,-0.006521,-0.006144,-0.008246,-0.011330,-0.011394,input/cremad/AudioWAV/1009_IOM_NEU_XX.wav,0.054391


In [5]:
sample_audio_data = np.array(sample_data.iloc[:,:-2])
sample_audio_data

array([[-0.01991875, -0.01807425, -0.01652197, ..., -0.03285718,
        -0.04676997, -0.05643881],
       [-0.0665344 , -0.06301752, -0.06548747, ..., -0.19164746,
        -0.23952487, -0.25656232],
       [ 0.01835893,  0.03975253,  0.06311458, ..., -0.0766595 ,
         0.0356272 ,  0.13745405],
       ...,
       [ 0.00966362, -0.00664271, -0.01405391, ..., -0.00081428,
        -0.00088602, -0.0009216 ],
       [ 0.00166954,  0.00117276,  0.00074541, ..., -0.00824648,
        -0.01133044, -0.0113936 ],
       [ 0.00312834,  0.00387238,  0.00473753, ...,  0.00550377,
         0.00466317,  0.0059031 ]])

In [6]:
def adjust_length(data, min_length):
    midpoint = int(len(data)/2)
    left_index = midpoint - int(min_length/2)
    return data[left_index:left_index+min_length]

In [7]:
def divide_into_frames(data,frame_length=2048, hop_length = 512):
    output = []
    n_frame = math.ceil(len(data)/hop_length)
    for i in range(n_frame-1):
        start_index = i * hop_length
        if start_index + frame_length > len(data):
            end_index = len(data)-hop_length*(n_frame-i)
            output.append(data[end_index-frame_length:end_index])
        else:
            output.append(data[start_index:start_index+frame_length])
    output.append(data[frame_length*(-1):])
    return np.array(output)

In [8]:
input_data = []
for i in range(len(sample_audio_data)):
    print(i)
    input_data.append(divide_into_frames(sample_audio_data[i]))
len(input_data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084


3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649


3650

In [9]:
input_data = np.array(input_data)
input_data = input_data.reshape(3650*56,2048)

In [114]:

# min_file_length = -1
# for sample_path in sample_path_and_label.Path:
#     data, sample_rate = librosa.load(sample_path)
#     if min_file_length < 0:
#         min_file_length = len(data)
#     else:
#         min_file_length = min(min_file_length, len(data))
# min_file_length

28402

In [135]:
# sample_audio_data = []
# zcr_list = []
# count = 0
# for sample_path in sample_path_and_label.Path:
#     data, sample_rate = librosa.load(sample_path)
#     adjusted_data = adjust_length(data,min_file_length)
#     sample_audio_data.append(adjusted_data)
#     zcr_list.append(np.mean(librosa.feature.zero_crossing_rate(y=adjusted_data).T, axis=0))
#     count += 1
#     if(count % 100 == 0):
#         print(str(count) + " / 3650")

100 / 3650
200 / 3650
300 / 3650
400 / 3650
500 / 3650
600 / 3650
700 / 3650
800 / 3650
900 / 3650
1000 / 3650
1100 / 3650
1200 / 3650
1300 / 3650
1400 / 3650
1500 / 3650
1600 / 3650
1700 / 3650
1800 / 3650
1900 / 3650
2000 / 3650
2100 / 3650
2200 / 3650
2300 / 3650
2400 / 3650
2500 / 3650
2600 / 3650
2700 / 3650
2800 / 3650
2900 / 3650
3000 / 3650
3100 / 3650
3200 / 3650
3300 / 3650
3400 / 3650
3500 / 3650
3600 / 3650


[array([-0.01991875, -0.01807425, -0.01652197, ..., -0.03285718,
        -0.04676997, -0.05643881], dtype=float32),
 array([-0.0665344 , -0.06301752, -0.06548747, ..., -0.19164746,
        -0.23952487, -0.25656232], dtype=float32),
 array([ 0.01835893,  0.03975253,  0.06311458, ..., -0.0766595 ,
         0.0356272 ,  0.13745405], dtype=float32),
 array([ 0.07308779,  0.06895027,  0.09250633, ..., -0.06927002,
        -0.08658554, -0.10061433], dtype=float32),
 array([-0.03515071,  0.02518957,  0.06201823, ..., -0.21344931,
        -0.04230656,  0.17547071], dtype=float32),
 array([-0.01367271, -0.00719742, -0.00578209, ...,  0.00652384,
         0.01971919,  0.03059095], dtype=float32),
 array([-0.00826108, -0.00764633,  0.01017111, ..., -0.00077834,
         0.00138452,  0.0009736 ], dtype=float32),
 array([ 0.00614743,  0.03099286,  0.01946602, ...,  0.05705999,
         0.03491644, -0.03950475], dtype=float32),
 array([-0.02706666, -0.02660906, -0.02547335, ..., -0.00485049,
       

In [10]:
zcr_list = []
count = 0
for i in input_data:
    zcr_list.append(librosa.feature.zero_crossing_rate(y=i,frame_length=4096,hop_length=4096).T)
    count += 1
    if(count % 100 == 0):
        print(str(count) + " / "+str(3650*56))

100 / 204400
200 / 204400
300 / 204400
400 / 204400
500 / 204400
600 / 204400
700 / 204400
800 / 204400
900 / 204400
1000 / 204400
1100 / 204400
1200 / 204400
1300 / 204400
1400 / 204400
1500 / 204400
1600 / 204400
1700 / 204400
1800 / 204400
1900 / 204400
2000 / 204400
2100 / 204400
2200 / 204400
2300 / 204400
2400 / 204400
2500 / 204400
2600 / 204400
2700 / 204400
2800 / 204400
2900 / 204400
3000 / 204400
3100 / 204400
3200 / 204400
3300 / 204400
3400 / 204400
3500 / 204400
3600 / 204400
3700 / 204400
3800 / 204400
3900 / 204400
4000 / 204400
4100 / 204400
4200 / 204400
4300 / 204400
4400 / 204400
4500 / 204400
4600 / 204400
4700 / 204400
4800 / 204400
4900 / 204400
5000 / 204400
5100 / 204400
5200 / 204400
5300 / 204400
5400 / 204400
5500 / 204400
5600 / 204400
5700 / 204400
5800 / 204400
5900 / 204400
6000 / 204400
6100 / 204400
6200 / 204400
6300 / 204400
6400 / 204400
6500 / 204400
6600 / 204400
6700 / 204400
6800 / 204400
6900 / 204400
7000 / 204400
7100 / 204400
7200 / 204400
7

56100 / 204400
56200 / 204400
56300 / 204400
56400 / 204400
56500 / 204400
56600 / 204400
56700 / 204400
56800 / 204400
56900 / 204400
57000 / 204400
57100 / 204400
57200 / 204400
57300 / 204400
57400 / 204400
57500 / 204400
57600 / 204400
57700 / 204400
57800 / 204400
57900 / 204400
58000 / 204400
58100 / 204400
58200 / 204400
58300 / 204400
58400 / 204400
58500 / 204400
58600 / 204400
58700 / 204400
58800 / 204400
58900 / 204400
59000 / 204400
59100 / 204400
59200 / 204400
59300 / 204400
59400 / 204400
59500 / 204400
59600 / 204400
59700 / 204400
59800 / 204400
59900 / 204400
60000 / 204400
60100 / 204400
60200 / 204400
60300 / 204400
60400 / 204400
60500 / 204400
60600 / 204400
60700 / 204400
60800 / 204400
60900 / 204400
61000 / 204400
61100 / 204400
61200 / 204400
61300 / 204400
61400 / 204400
61500 / 204400
61600 / 204400
61700 / 204400
61800 / 204400
61900 / 204400
62000 / 204400
62100 / 204400
62200 / 204400
62300 / 204400
62400 / 204400
62500 / 204400
62600 / 204400
62700 / 20

110400 / 204400
110500 / 204400
110600 / 204400
110700 / 204400
110800 / 204400
110900 / 204400
111000 / 204400
111100 / 204400
111200 / 204400
111300 / 204400
111400 / 204400
111500 / 204400
111600 / 204400
111700 / 204400
111800 / 204400
111900 / 204400
112000 / 204400
112100 / 204400
112200 / 204400
112300 / 204400
112400 / 204400
112500 / 204400
112600 / 204400
112700 / 204400
112800 / 204400
112900 / 204400
113000 / 204400
113100 / 204400
113200 / 204400
113300 / 204400
113400 / 204400
113500 / 204400
113600 / 204400
113700 / 204400
113800 / 204400
113900 / 204400
114000 / 204400
114100 / 204400
114200 / 204400
114300 / 204400
114400 / 204400
114500 / 204400
114600 / 204400
114700 / 204400
114800 / 204400
114900 / 204400
115000 / 204400
115100 / 204400
115200 / 204400
115300 / 204400
115400 / 204400
115500 / 204400
115600 / 204400
115700 / 204400
115800 / 204400
115900 / 204400
116000 / 204400
116100 / 204400
116200 / 204400
116300 / 204400
116400 / 204400
116500 / 204400
116600 /

162100 / 204400
162200 / 204400
162300 / 204400
162400 / 204400
162500 / 204400
162600 / 204400
162700 / 204400
162800 / 204400
162900 / 204400
163000 / 204400
163100 / 204400
163200 / 204400
163300 / 204400
163400 / 204400
163500 / 204400
163600 / 204400
163700 / 204400
163800 / 204400
163900 / 204400
164000 / 204400
164100 / 204400
164200 / 204400
164300 / 204400
164400 / 204400
164500 / 204400
164600 / 204400
164700 / 204400
164800 / 204400
164900 / 204400
165000 / 204400
165100 / 204400
165200 / 204400
165300 / 204400
165400 / 204400
165500 / 204400
165600 / 204400
165700 / 204400
165800 / 204400
165900 / 204400
166000 / 204400
166100 / 204400
166200 / 204400
166300 / 204400
166400 / 204400
166500 / 204400
166600 / 204400
166700 / 204400
166800 / 204400
166900 / 204400
167000 / 204400
167100 / 204400
167200 / 204400
167300 / 204400
167400 / 204400
167500 / 204400
167600 / 204400
167700 / 204400
167800 / 204400
167900 / 204400
168000 / 204400
168100 / 204400
168200 / 204400
168300 /

In [11]:
zcr_list = np.array(zcr_list)
zcr_list = zcr_list.reshape(204400,1)
zcr_list.shape

(204400, 1)

## Train FINs

### ZCR

In [19]:
#input: path -> array(in fixed length) -> dense neural network -> output(zcr)

#1. Load data
ZCR_X = input_data
ZCR_y = zcr_list

In [27]:
#model
model = Sequential()
model.add(Dense(1024, input_dim=2048, activation='softsign'))
model.add(Dense(1024, activation='softsign'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(optimizer='adam',
    loss='mse',
    metrics=['mse'])

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_11 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_12 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_13 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_14 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_15 (Dense)             (None, 32)               

In [20]:
# splitting data
zcr_x_train, zcr_x_test, zcr_y_train, zcr_y_test = train_test_split(ZCR_X, ZCR_y, random_state=0, shuffle=True)
zcr_x_train.shape, zcr_y_train.shape, zcr_x_test.shape, zcr_y_test.shape

((153300, 2048), (153300, 1), (51100, 2048), (51100, 1))

In [29]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
history = model.fit(zcr_x_train, zcr_y_train, batch_size=64, epochs=50, validation_data=(zcr_x_test, zcr_y_test), callbacks=[rlrp])

Train on 153300 samples, validate on 51100 samples
Epoch 1/50
153300/153300 [==============================] - 162s 1ms/step - loss: 0.0021 - mse: 0.0021 - val_loss: 0.0017 - val_mse: 0.0017
Epoch 2/50
153300/153300 [==============================] - 173s 1ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0010 - val_mse: 0.0010
Epoch 3/50
153300/153300 [==============================] - 178s 1ms/step - loss: 8.9023e-04 - mse: 8.9023e-04 - val_loss: 9.2746e-04 - val_mse: 9.2746e-04
Epoch 4/50
153300/153300 [==============================] - 173s 1ms/step - loss: 7.1506e-04 - mse: 7.1506e-04 - val_loss: 7.9931e-04 - val_mse: 7.9931e-04
Epoch 5/50
153300/153300 [==============================] - 169s 1ms/step - loss: 6.1050e-04 - mse: 6.1050e-04 - val_loss: 8.4236e-04 - val_mse: 8.4236e-04
Epoch 6/50
153300/153300 [==============================] - 174s 1ms/step - loss: 5.3483e-04 - mse: 5.3483e-04 - val_loss: 6.3488e-04 - val_mse: 6.3488e-04
Epoch 7/50
153300/153300 [===================

In [30]:
history.history

{'val_loss': [0.0016665159312392167,
  0.0010329062879767174,
  0.0009274583366068915,
  0.0007993071904559046,
  0.000842361730913433,
  0.000634877872279683,
  0.0006593360770776241,
  0.0006299401469548723,
  0.000648648655319991,
  0.0005826712972616663,
  0.0005513661506666881,
  0.0005540366084410567,
  0.0005218253929829571,
  0.0005245091302371393,
  0.000517110448239969,
  0.000522904491349107,
  0.000513129414980976,
  0.0005074777741439881,
  0.0005080976407385363,
  0.0005080950226285723,
  0.000509705580755022,
  0.0005087432237829532,
  0.0005081083584205957,
  0.0005092827739069191,
  0.0005092411326178862,
  0.0005087096100525452,
  0.0005085963047443982,
  0.0005085832943275038,
  0.0005086069926004419,
  0.0005086322399854128,
  0.0005086669163279958,
  0.0005086526551184567,
  0.0005086463577216109,
  0.0005086253095056438,
  0.0005086453574428922,
  0.0005086265362634226,
  0.0005086456889395079,
  0.000508597667102106,
  0.0005086757547341455,
  0.00050863151071746

In [31]:
pred = model.predict(zcr_x_test)

In [32]:
r2_score(zcr_y_test, pred)

0.7535884199792946

In [ ]:
print("Accuracy of our model on test data : " , model.evaluate(zcr_x_test,zcr_y_test)[1]*100 , "%")

epochs = [i for i in range(50)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['mean_squared_error']
train_loss = history.history['loss']
test_acc = history.history['val_mean_squared_error']
test_loss = history.history['val_loss']

fig.set_size_inches(20,6)
ax[0].plot(epochs , train_loss , label = 'Training Loss')
ax[0].plot(epochs , test_loss , label = 'Testing Loss')
ax[0].set_title('Training & Testing Loss')
ax[0].legend()
ax[0].set_xlabel("Epochs")

ax[1].plot(epochs , train_acc , label = 'Training Accuracy')
ax[1].plot(epochs , test_acc , label = 'Testing Accuracy')
ax[1].set_title('Training & Testing Accuracy')
ax[1].legend()
ax[1].set_xlabel("Epochs")
plt.show()

## Save FINs

In [26]:
model.save("FIN_zcr")

## Ensemble FINs

### 1. Fix FIN weights

### 2. Free FIN weights